In [1]:
#import the packages required
import pandas as pd
import numpy as np
import re
from scipy import sparse
from sklearn import preprocessing, linear_model
import sklearn.feature_extraction.text as txt

In [2]:
#read test and train dataset
train = pd.read_csv('TrainingData.csv')
test = pd.read_csv('TestData.csv')
sample = pd.read_csv('SubmissionFormat.csv')


C:\Users\derri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#printing train dataset
train.head()


,Unnamed: 0,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
0,134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,...,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
1,206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,...,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
2,326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,...,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
3,364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,...,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
4,47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,...,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


In [4]:
#printing test dataset
test.head()

,Unnamed: 0,Object_Description,Program_Description,SubFund_Description,Job_Title_Description,Facility_or_Department,Sub_Object_Description,Location_Description,FTE,Function_Description,Position_Extra,Text_4,Total,Text_2,Text_3,Fund_Description,Text_1
0,180042,Student Meals/Room/Other,Basic Educational Services,NaN,NaN,NaN,Line Item that is paid with Campus' money,School,NaN,Instruction,NaN,NaN,3999.910000,NaN,NaN,General Fund,NaN
1,28872,Extra Duty/Signing Bonus Pay,Undistributed,NaN,CHEERLEADER DIR,NaN,General,School,NaN,Cocurricular & Extra Curricular Activities,NaN,NaN,3447.320213,NaN,NaN,General Fund,NaN
2,186915,Professional Salaries,Bilingual Education,NaN,T-EL 1ST BIL,NaN,General,School,1.0,Instruction,NaN,NaN,52738.780869,NaN,NaN,General Fund,NaN
3,412396,Professional Salaries,Bilingual Education,NaN,T-EL 2ND BIL,NaN,General,School,1.0,Instruction,NaN,NaN,69729.263191,NaN,NaN,General Fund,NaN
4,427740,Salaries for Support Personnel,Undistributed,NaN,CLERK III- SCH,NaN,General,School,1.0,School Leadership,NaN,NaN,29492.834215,NaN,NaN,General Fund,NaN


In [5]:

training_data = train[['FTE','Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4', 'Total']]

cols = ['Function','Object_Type','Operating_Status','Position_Type','Pre_K', 'Reporting',
                'Sharing','Student_Type', 'Use']

labels = train[cols]

it=['Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4']
test_data = test[['FTE','Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4', 'Total']]

In [6]:
#data cleaning
def clean(s):
    try:
        return " ".join(re.findall(r'\w+', s)).lower()
    except:
        return " ".join(re.findall(r'\w+', "no_text")).lower()


In [7]:
#encoding labels

label_encoder = preprocessing.LabelEncoder()
for i in cols:
    labels[i] = label_encoder.fit_transform(np.array(labels[i]))
    

C:\Users\derri\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
#drop columns which has non text values
training_data = training_data.drop('FTE', axis = 1)
training_data = training_data.drop('Total', axis = 1)

test_data = test_data.drop('FTE', axis = 1)
test_data = test_data.drop('Total', axis = 1)



for i in it:
        training_data[i] = training_data[i].apply(clean)
for i in it:
        test_data[i] = test_data[i].apply(clean)
        


In [9]:
#printing training data
training_data.head()



,Facility_or_Department,Function_Description,Fund_Description,Job_Title_Description,Location_Description,Object_Description,Position_Extra,Program_Description,SubFund_Description,Sub_Object_Description,Text_1,Text_2,Text_3,Text_4
0,no_text,no_text,general fund,teacher elementary,no_text,no_text,kindergarten,kindergarten,no_text,no_text,no_text,no_text,no_text,no_text
1,no_text,rgn gob,no_text,blank,no_text,contractor services,undesignated,building improvement services,building fund,no_text,building improvement services,bond expenditures,regular,no_text
2,no_text,no_text,general purpose school,tcher 2nd grade,no_text,personal services teachers,teacher,instruction regular,no_text,no_text,no_text,no_text,no_text,regular instruction
3,no_text,unalloc budgets schools,no_text,teacher short term sub,no_text,employee benefits,professional instructional,general middle junior high sch,general fund,no_text,regular instruction,teacher subs,regular,no_text
4,no_text,non project,no_text,teacher secondary high,no_text,teacher coverage for teacher,professional instructional,general high school education,general fund,no_text,regular instruction,teacher subs,alternative,no_text


In [10]:
#printing testing data
test_data.head()

,Facility_or_Department,Function_Description,Fund_Description,Job_Title_Description,Location_Description,Object_Description,Position_Extra,Program_Description,SubFund_Description,Sub_Object_Description,Text_1,Text_2,Text_3,Text_4
0,no_text,instruction,general fund,no_text,school,student meals room other,no_text,basic educational services,no_text,line item that is paid with campus money,no_text,no_text,no_text,no_text
1,no_text,cocurricular extra curricular activities,general fund,cheerleader dir,school,extra duty signing bonus pay,no_text,undistributed,no_text,general,no_text,no_text,no_text,no_text
2,no_text,instruction,general fund,t el 1st bil,school,professional salaries,no_text,bilingual education,no_text,general,no_text,no_text,no_text,no_text
3,no_text,instruction,general fund,t el 2nd bil,school,professional salaries,no_text,bilingual education,no_text,general,no_text,no_text,no_text,no_text
4,no_text,school leadership,general fund,clerk iii sch,school,salaries for support personnel,no_text,undistributed,no_text,general,no_text,no_text,no_text,no_text


In [11]:
#combine all column of the dataset
training_data["combined"] = [' '.join(row) for row in training_data[training_data.columns].values]
test_data["combined"] = [' '.join(row) for row in test_data[test_data.columns].values]


In [12]:
#combined columns of train dataset
training_data.head()


,Facility_or_Department,Function_Description,Fund_Description,Job_Title_Description,Location_Description,Object_Description,Position_Extra,Program_Description,SubFund_Description,Sub_Object_Description,Text_1,Text_2,Text_3,Text_4,combined
0,no_text,no_text,general fund,teacher elementary,no_text,no_text,kindergarten,kindergarten,no_text,no_text,no_text,no_text,no_text,no_text,no_text no_text general fund teacher elementar...
1,no_text,rgn gob,no_text,blank,no_text,contractor services,undesignated,building improvement services,building fund,no_text,building improvement services,bond expenditures,regular,no_text,no_text rgn gob no_text blank no_text contract...
2,no_text,no_text,general purpose school,tcher 2nd grade,no_text,personal services teachers,teacher,instruction regular,no_text,no_text,no_text,no_text,no_text,regular instruction,no_text no_text general purpose school tcher 2...
3,no_text,unalloc budgets schools,no_text,teacher short term sub,no_text,employee benefits,professional instructional,general middle junior high sch,general fund,no_text,regular instruction,teacher subs,regular,no_text,no_text unalloc budgets schools no_text teache...
4,no_text,non project,no_text,teacher secondary high,no_text,teacher coverage for teacher,professional instructional,general high school education,general fund,no_text,regular instruction,teacher subs,alternative,no_text,no_text non project no_text teacher secondary ...


In [13]:
#combined columns of train dataset
test_data.head()


,Facility_or_Department,Function_Description,Fund_Description,Job_Title_Description,Location_Description,Object_Description,Position_Extra,Program_Description,SubFund_Description,Sub_Object_Description,Text_1,Text_2,Text_3,Text_4,combined
0,no_text,instruction,general fund,no_text,school,student meals room other,no_text,basic educational services,no_text,line item that is paid with campus money,no_text,no_text,no_text,no_text,no_text instruction general fund no_text schoo...
1,no_text,cocurricular extra curricular activities,general fund,cheerleader dir,school,extra duty signing bonus pay,no_text,undistributed,no_text,general,no_text,no_text,no_text,no_text,no_text cocurricular extra curricular activiti...
2,no_text,instruction,general fund,t el 1st bil,school,professional salaries,no_text,bilingual education,no_text,general,no_text,no_text,no_text,no_text,no_text instruction general fund t el 1st bil ...
3,no_text,instruction,general fund,t el 2nd bil,school,professional salaries,no_text,bilingual education,no_text,general,no_text,no_text,no_text,no_text,no_text instruction general fund t el 2nd bil ...
4,no_text,school leadership,general fund,clerk iii sch,school,salaries for support personnel,no_text,undistributed,no_text,general,no_text,no_text,no_text,no_text,no_text school leadership general fund clerk i...


In [14]:
#conversion of text data values to numbers using hashing technique
tfidf = txt.TfidfVectorizer(ngram_range=(1,2), max_df=1.0, min_df=10)
hsv = txt.HashingVectorizer()


tfidf.fit(training_data['combined'])
hsv.fit(test_data['combined'])

X_tfidf = tfidf.transform(training_data['combined'])
X_test_tfidf = tfidf.transform(test_data['combined'])

X_hsv = hsv.transform(training_data['combined'])
X_test_hsv = hsv.transform(test_data['combined'])

X = sparse.hstack((X_hsv, X_tfidf))
X_test = sparse.hstack((X_test_hsv, X_test_tfidf))


In [15]:
prediction1 = []
prediction2 = []
prediction3 = []
prediction4 = []

In [17]:
#SGD model training and prediction
for i in range(len(cols)):
    print ("label = ", i)
    sgd1 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, alpha = 0.000001)
    sgd2 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, penalty = 'l1')
    sgd3 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, penalty = 'l2')
    sgd4 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, penalty = 'elasticnet')
    sgd1.fit(X, labels.iloc[:,i])
    sgd2.fit(X, labels.iloc[:,i])
    sgd3.fit(X, labels.iloc[:,i])
    sgd4.fit(X, labels.iloc[:,i])
    if i == 0:
        prediction1 = sgd1.predict_proba(X_test)
        prediction2 = sgd2.predict_proba(X_test)
        prediction3 = sgd3.predict_proba(X_test)
        prediction4 = sgd4.predict_proba(X_test)
    else:
        print(len(prediction1),len(sgd1.predict_proba(X_test)))
        prediction1 = np.hstack((prediction1,sgd1.predict_proba(X_test)))
        prediction2 = np.hstack((prediction2,sgd2.predict_proba(X_test)))
        prediction3 = np.hstack((prediction3,sgd3.predict_proba(X_test)))
        prediction4 = np.hstack((prediction4,sgd4.predict_proba(X_test)))
        
preds = (prediction1 + prediction2 + prediction3 + prediction4)/4.0

for i in range(1, len(sample.columns)):
    colname = sample.columns[i]
    sample[str(colname)] = preds[:,i-1]

sample.to_csv('submitpreds1.csv', index = False)

label =  0
label =  1
50064 50064
label =  2
50064 50064
label =  3
50064 50064
label =  4
50064 50064
label =  5
50064 50064
label =  6
50064 50064
label =  7
50064 50064
label =  8
50064 50064


In [18]:
#printing submission.csv 
submission = pd.read_csv('submitpreds1.csv')
submission.head()

,Unnamed: 0,Function__Aides Compensation,Function__Career & Academic Counseling,Function__Communications,Function__Curriculum Development,Function__Data Processing & Information Services,Function__Development & Fundraising,Function__Enrichment,Function__Extended Time & Tutoring,Function__Facilities & Maintenance,...,Student_Type__Special Education,Student_Type__Unspecified,Use__Business Services,Use__ISPD,Use__Instruction,Use__Leadership,Use__NO_LABEL,Use__O&M,Use__Pupil Services & Enrichment,Use__Untracked Budget Set-Aside
0,180042,0.027779,0.006721,0.000772,0.005231,0.006322,0.000531,0.014861,0.005926,0.017196,...,0.017527,0.509301,0.004311,0.014487,0.652988,0.017007,0.255455,0.022679,0.030923,0.002150
1,28872,0.019007,0.009714,0.001267,0.011963,0.010729,0.000659,0.541902,0.036214,0.028675,...,0.014859,0.901644,0.021773,0.035369,0.165606,0.155260,0.078792,0.026552,0.515594,0.001054
2,186915,0.057329,0.007395,0.001044,0.011552,0.007577,0.000578,0.015907,0.005713,0.013709,...,0.015882,0.458161,0.006791,0.021169,0.735962,0.026510,0.163786,0.008251,0.036831,0.000699
3,412396,0.056444,0.007219,0.001017,0.011219,0.007371,0.000562,0.015416,0.005550,0.013301,...,0.015681,0.454495,0.006666,0.020465,0.752497,0.026208,0.149101,0.008141,0.036230,0.000691
4,427740,0.020501,0.007893,0.000646,0.003891,0.005204,0.000324,0.016517,0.001576,0.015481,...,0.016228,0.938852,0.014696,0.069904,0.101404,0.702213,0.029490,0.040989,0.040729,0.000574
